In [11]:
!pip install yfinance sqlalchemy statsmodels lxml

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 46.1 MB/s eta 0:00:00

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import requests
import sqlite3
from sqlalchemy import create_engine
from datetime import datetime, timedelta
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}
response = requests.get(url, headers=headers)
tables = pd.read_html(response.text)
sp500_table = tables[0]

print("Successfully extracted table from Wikipedia")
print(f"Table shape: {sp500_table.shape}")
print(f"Columns: {list(sp500_table.columns)}\n")

Successfully extracted table from Wikipedia
Table shape: (503, 8)
Columns: ['Symbol', 'Security', 'GICS Sector', 'GICS Sub-Industry', 'Headquarters Location', 'Date added', 'CIK', 'Founded']



In [4]:
# Extract tickers and clean them (Yahoo Finance uses '-' for dots, e.g., 'BRK.B' -> 'BRK-B')
tickers = sp500_table['Symbol'].str.replace('.', '-').tolist()
print(f"Total number of tickers extracted: {len(tickers)}")
print(f"First 10 tickers: {tickers[:10]}\n")

Total number of tickers extracted: 503
First 10 tickers: ['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ADBE', 'AMD', 'AES', 'AFL', 'A']



In [5]:
end_date = datetime.now()
start_date = end_date - timedelta(days=3*365)

print(f"Date range: {start_date.date()} to {end_date.date()}")

Date range: 2022-10-19 to 2025-10-18


In [8]:
# Download S&P 500 Index data
sp500_index = yf.download('^GSPC', start=start_date, end=end_date, progress=False)
print(f"S&P 500 Index downloaded: {len(sp500_index)} days of data\n")

S&P 500 Index downloaded: 752 days of data



In [7]:
stock_data = yf.download(tickers, start=start_date, end=end_date, group_by='ticker', progress=True)

print(f"Downloaded: {stock_data.shape}")

[*********************100%***********************]  503 of 503 completed


Downloaded: (752, 2515)


In [9]:
# Extract 'Close' prices
adj_close_prices = stock_data.xs('Close', level=1, axis=1)

# Calculate stock returns (Reference: PCA_example.py)
returns = adj_close_prices.pct_change().dropna()

# Calculate S&P 500 returns (Reference: PCA_example.py)
sp500_returns = sp500_index['Close'].pct_change().dropna()

print(f"Stock returns calculated: {returns.shape}")
print(f"S&P 500 returns calculated: {sp500_returns.shape}")

Stock returns calculated: (391, 503)
S&P 500 returns calculated: (751, 1)


In [10]:
db_name = 'sp500_data.db'
engine = create_engine(f'sqlite:///{db_name}')

# Store data in database (Reference: PCA_example.py for SQL interaction)
adj_close_prices.to_sql('adj_close_prices', engine, if_exists='replace', index=True)
returns.to_sql('returns', engine, if_exists='replace', index=True)
sp500_index.to_sql('sp500_index', engine, if_exists='replace', index=True)
sp500_returns.to_sql('sp500_returns', engine, if_exists='replace', index=True)

print(f"All data successfully stored in '{db_name}'")

All data successfully stored in 'sp500_data.db'


In [12]:
# Standardize returns (Reference: PCA_simulation.py, PCA_example.py)
scaler = StandardScaler()
returns_standardized = scaler.fit_transform(returns)
print(f"Data standardized: {returns_standardized.shape}")

# Apply PCA with 5 components (Reference: PCA_simulation.py, PCA_example.py)
pca = PCA(n_components=5)
principal_components = pca.fit_transform(returns_standardized)
print(f"PCA completed: {principal_components.shape}\n")

# Variance explained
variance_explained = pca.explained_variance_ratio_
cumulative_variance = np.cumsum(variance_explained)

print("Variance Explained by First Five Components:")
print("-" * 50)
for i in range(5):
    print(f"  PC{i+1}: {variance_explained[i]*100:>6.2f}% (Cumulative: {cumulative_variance[i]*100:.2f}%)")
print("-" * 50)
print(f"Total Variance Explained by 5 factors: {cumulative_variance[-1]*100:.2f}%")

# Create a DataFrame for the PC "factor returns"
pc_returns = pd.DataFrame(
    principal_components,
    index=returns.index,
    columns=[f'PC{i+1}' for i in range(5)]
)
print(f"\nPC returns DataFrame created: {pc_returns.shape}")

Data standardized: (391, 503)
PCA completed: (391, 5)

Variance Explained by First Five Components:
--------------------------------------------------
  PC1:  28.71% (Cumulative: 28.71%)
  PC2:   8.76% (Cumulative: 37.47%)
  PC3:   3.61% (Cumulative: 41.07%)
  PC4:   2.52% (Cumulative: 43.60%)
  PC5:   1.96% (Cumulative: 45.55%)
--------------------------------------------------
Total Variance Explained by 5 factors: 45.55%

PC returns DataFrame created: (391, 5)


In [13]:
# Align dates
common_dates = pc_returns.index.intersection(sp500_returns.index)
pc_returns_aligned = pc_returns.loc[common_dates]
sp500_returns_aligned = sp500_returns.loc[common_dates]

print(f"Aligned data for regression: {len(common_dates)} dates\n")

# Run regressions for each PC
regression_results = {}
X = sm.add_constant(sp500_returns_aligned.values)  # Add intercept

for i in range(1, 6):
    pc_name = f'PC{i}'
    
    # Prepare regression data
    y = pc_returns_aligned[pc_name].values

    # Fit OLS model (Reference: 612-CrossSectionalAnalysis.pdf, Page 9 )
    model = sm.OLS(y, X).fit()

    # Store results
    regression_results[pc_name] = {
        'alpha (β0)': model.params[0],
        'beta (β1)': model.params[1],
        'alpha_pvalue': model.pvalues[0],
        'beta_pvalue': model.pvalues[1],
        'r_squared': model.rsquared
    }

# Display results in a summary table
summary_df = pd.DataFrame(regression_results).T
summary_df.index.name = "Factor (PC)"
print(summary_df.to_string(float_format="%.4f"))

Aligned data for regression: 391 dates

             alpha (β0)  beta (β1)  alpha_pvalue  beta_pvalue  r_squared
Factor (PC)                                                             
PC1             -0.6633   992.8950        0.0197       0.0000     0.7848
PC2              0.1226  -183.4429        0.7037       0.0000     0.0878
PC3             -0.0360    53.8345        0.8669       0.0073     0.0184
PC4              0.0391   -58.4994        0.8265       0.0005     0.0310
PC5              0.0018    -2.6861        0.9910       0.8564     0.0001


Q4:
These 5 components capture 45.55% of the total ups and downs:

PC1: 28.71%
PC2: 8.76%
PC3: 3.61%
PC4: 2.52%
PC5: 1.96%

PC1: High beta (993), r_squared 78% – Alpha is negative but small.
PC2: Negative beta (-183), r_squared 9% – moves opposite market, maybe a hedge.
PC3: Positive beta (54), r_squared 2% – weak market link.
PC4: Negative beta (-58), r_squared 3% – another contrarian.
PC5: Near-zero beta, r_squared 0% – random noise

Q5 (reference - DimensionalityReduction.pdf):
PCA's' goal of the PCA is to maximize variance.
FA's find directions for sum of wi * xi maximum variance.
PCA is preferred in this case since it directly maximizes explained variance. Also PCA is simpler than FA, since FA might overcomplicate things by assuming latent factors that may not exist or fit the market data well